In [11]:
import csv
from enum import Enum
import numpy as np

from model.neuron_metadata import NeuronMetadataCollection

neuron_metadata_collection = NeuronMetadataCollection.load_from_chem_json(get_data_file_abs_path('chem.json'))

import project_path
from model.data_accessor import get_data_file_abs_path

connectome_file = 'herm_full_edgelist.csv'

# key = conn_spec = (from, to)
# value = total weight
conn_spec_to_weight_chem = {}
conn_spec_to_weight_gap = {}

cook_neurons = set()
with open(get_data_file_abs_path(connectome_file), newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  i = 0
  for row in reader:
    source = row['Source'].upper().strip()
    target = row['Target'].upper().strip()
    weight = row['Weight'].upper().strip()
    conn_type = row['Type'].upper().strip()
    conn_spec = (source, target)
    cook_neurons.add(source)
    cook_neurons.add(target)
    
    conn_spec_to_weight = None
    if conn_type == "CHEMICAL":
      conn_spec_to_weight = conn_spec_to_weight_chem
    elif conn_type == "ELECTRICAL":
      conn_spec_to_weight = conn_spec_to_weight_gap
    else:
      raise Exception("Invalid connection type: " + conn_type)
      
    if conn_spec in conn_spec_to_weight:
      raise Exception("Duplicate entry exists for %s. Previous value is %d, new value is %d" %\
                      (conn_type, conn_spec_to_weight[conn_spec], weight))
    conn_spec_to_weight[conn_spec] = weight


# Neuron set differences

In [24]:
# Cook has 448 neurons, Varshney 279
varshney_neurons = neuron_metadata_collection.name_to_id.keys()
print("Cook has %d neurons, Varshney %d" % (len(cook_neurons), len(varshney_neurons)))

# All neurons in varshney found in cook
varshney_diff_cook = varshney_neurons - cook_neurons
print("In varshney, but not in cook: (%d) %s" % (len(varshney_diff_cook), varshney_diff_cook))

# 169 neurons in cook are not found in varshney.
cook_diff_varshney = cook_neurons - varshney_neurons
print("In cook, but not in varshney: (%d) %s" % (len(cook_diff_varshney), sorted(cook_diff_varshney)))


Cook has 448 neurons, Varshney 279
In varshney, but not in cook: (0) set()
In cook, but not in varshney: (169) ['ANAL', 'DBWML1', 'DBWML10', 'DBWML11', 'DBWML12', 'DBWML13', 'DBWML14', 'DBWML15', 'DBWML16', 'DBWML17', 'DBWML18', 'DBWML19', 'DBWML2', 'DBWML20', 'DBWML21', 'DBWML22', 'DBWML23', 'DBWML24', 'DBWML3', 'DBWML4', 'DBWML5', 'DBWML6', 'DBWML7', 'DBWML8', 'DBWML9', 'DBWMR1', 'DBWMR10', 'DBWMR11', 'DBWMR12', 'DBWMR13', 'DBWMR14', 'DBWMR15', 'DBWMR16', 'DBWMR17', 'DBWMR18', 'DBWMR19', 'DBWMR2', 'DBWMR20', 'DBWMR21', 'DBWMR22', 'DBWMR23', 'DBWMR24', 'DBWMR3', 'DBWMR4', 'DBWMR5', 'DBWMR6', 'DBWMR7', 'DBWMR8', 'DBWMR9', 'HYP', 'I1L', 'I1R', 'I2L', 'I2R', 'I3', 'I4', 'I5', 'I6', 'INTESTINE', 'INTL', 'INTR', 'M1', 'M2L', 'M2R', 'M3L', 'M3R', 'M4', 'M5', 'MC1DL', 'MC1DR', 'MC1V', 'MC2DL', 'MC2DR', 'MC2V', 'MC3DL', 'MC3DR', 'MC3V', 'MCL', 'MCR', 'MI', 'NSML', 'NSMR', 'PM1D', 'PM1VL', 'PM1VR', 'PM2D', 'PM2VL', 'PM2VR', 'PM3D', 'PM3VL', 'PM3VR', 'PM4D', 'PM4VL', 'PM4VR', 'PM5D', 'PM5VL', '

In [25]:
def build_connectome_matrix(conn_spec_to_weight, neuron_metadata_collection):
  N = neuron_metadata_collection.get_size()
  mat = np.zeros((N,N))
  
  for conn_spec, weight in conn_spec_to_weight.items():
    source, target = conn_spec
    source_id = neuron_metadata_collection.get_id_from_name(source)
    target_id = neuron_metadata_collection.get_id_from_name(target)
    # TODO: Should we really skip? Should we use the new N not from chem.json, but from cook 2019?
    if source_id < 0 or target_id < 0:
      continue
    # The existing Gg[i][j] means from neuron j to i.
    mat[target_id, source_id] = weight
  return mat
    
cook_Gg = build_connectome_matrix(conn_spec_to_weight_gap, neuron_metadata_collection)
cook_Gs = build_connectome_matrix(conn_spec_to_weight_chem, neuron_metadata_collection)